# Lab Rotation

In [1]:
import pandas as pd
import re as re
import numpy as np
import requests
from urllib.parse import urlparse
from time import sleep

In [2]:
# TODO 
github_token = "ADD_YOUR_GITHUB_TOKEN"

## Data Collection
@Misc{acl-ocl,
    author =       {Shaurya Rohatgi, Yanxia Qin, Benjamin Aw, Niranjana Unnithan, Min-Yen Kan},
    title =        {The ACL OCL Corpus: advancing Open science in Computational Linguistics},
    howpublished = {arXiv},
    year =         {2022},
    url =          {https://huggingface.co/datasets/ACL-OCL/ACL-OCL-Corpus}
}

In [3]:
df = pd.read_parquet('data/acl-publication-info.74k.v2.parquet')
df.head()

,acl_id,abstract,full_text,corpus_paper_id,pdf_hash,numcitedby,url,publisher,address,year,...,doi,number,volume,journal,editor,isbn,ENTRYTYPE,ID,language,note
0,O02-2002,There is a need to measure word similarity whe...,There is a need to measure word similarity whe...,18022704,0b09178ac8d17a92f16140365363d8df88c757d0,14,https://aclanthology.org/O02-2002,None,None,2002,...,None,None,None,None,None,None,inproceedings,chen-you-2002-study,None,None
1,L02-1310,None,None,8220988,8d5e31610bc82c2abc86bc20ceba684c97e66024,93,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,None,None,None,None,None,None,inproceedings,mihalcea-2002-bootstrapping,None,None
2,R13-1042,Thread disentanglement is the task of separati...,Thread disentanglement is the task of separati...,16703040,3eb736b17a5acb583b9a9bd99837427753632cdb,10,https://aclanthology.org/R13-1042,"INCOMA Ltd. Shoumen, BULGARIA","Hissar, Bulgaria",2013,...,None,None,None,None,None,None,inproceedings,jamison-gurevych-2013-headerless,None,None
3,W05-0819,"In this paper, we describe a word alignment al...","In this paper, we describe a word alignment al...",1215281,b20450f67116e59d1348fc472cfc09f96e348f55,15,https://aclanthology.org/W05-0819,Association for Computational Linguistics,"Ann Arbor, Michigan",2005,...,None,None,None,None,None,None,inproceedings,aswani-gaizauskas-2005-aligning,None,None
4,L02-1309,None,None,18078432,011e943b64a78dadc3440674419821ee080f0de3,12,http://www.lrec-conf.org/proceedings/lrec2002/...,European Language Resources Association (ELRA),"Las Palmas, Canary Islands - Spain",2002,...,None,None,None,None,None,None,inproceedings,suyaga-etal-2002-proposal,None,None


In [4]:
df = df.rename(columns={"acl_id": "paper_ident", # unique paper identifier
                   "url": "paper_url", # Paper online abstract page URL.
                   "author": "paper_author", # Author list.
                   "title": "paper_title", # Paper title
                   "journal": "paper_venue", # Venue abbreviation.
                   "year": "paper_year", # Publication year
                   "month": "paper_month", # Publication month.
                   "booktitle": "paper_booktitle", # BibTeX booktitle field.
                   "address": "paper_address", # BibTeX adress field
                   "publisher": "paper_publisher", # BibTeX publisher field      
                   "pages": "paper_pages", # BibTeX pages.
                   "full_text": "paper_text",
                   })

df = df.drop(columns=["abstract", "corpus_paper_id", "pdf_hash", "doi",
                              "numcitedby", "number", "volume",  
                              "editor", "isbn", "ENTRYTYPE","ID", "language", "note"])

In [5]:
df['error_download'] = df['paper_text'].apply(lambda x: not x.strip() if isinstance(x, str) else True)

df['error_download'].value_counts()

error_download
False    67414
True      5871
Name: count, dtype: int64

# BLEU
## Reproducibility
### BLEU identification

In [6]:
df_bleu_prelim = df.copy()
df_bleu_prelim["paper_bleu_prelim"] = df_bleu_prelim["paper_text"].str.contains("bleu", case=False)

df_bleu_prelim['paper_bleu_prelim'].value_counts()

paper_bleu_prelim
False    58333
True      9122
Name: count, dtype: int64

### Paper Review
#### BLEU Parameters

In [7]:
def extract_parameters(text):
    pattern = r"((?: -[a-z123](?: [a-z0-9.]{1,4})?){2,})"
    matches = re.findall(pattern, text)
    return matches[0] if matches else None

In [8]:
df_bleu_params = df_bleu_prelim.copy()
df_bleu_params["paper_bleu_params"] = df_bleu_params.apply(lambda row: extract_parameters(row['paper_text']) if row['paper_bleu_prelim'] else None, axis=1)

df_bleu_params['paper_bleu_params'].notna().sum()

54

#### BLEU Protocol

In [9]:
bleu_regex_protocol = {
    'ngrams': r"\bn-?grams?\b",
    'precision': r"\bn-?gram\sprecision\b",
    'clipping': r"\bclipping\b",
    'brevity_penalty': r"\bbrevity\spenalty\b|\bBP\b",
    'weights': r"\bweighting\sof\sn-?grams\b",
    'smoothing': r"\bsmoothing\b",
    'tokenization': r'\b(?:tokenized?|tokenizer|tokenization|pre-tokenized?|detokenized?)\b',
    'case_normalization': r'\b(?:case normalization|lowercased|case-insensitive|case sensitive)\b'
}

In [10]:
def search_terms_near_bleu(text, regex_dict):
    results = []
    for term, pattern in regex_dict.items():
        # Find all occurrences of 'bleu' (case insensitive)
        for match in re.finditer(r'bleu', text, re.IGNORECASE):
            start, end = match.start(), match.end()
            # Define a 500-character window around 'bleu'
            window_start, window_end = max(0, start - 500), min(len(text), end + 500)
            # Search for the term within this window
            if re.search(pattern, text[window_start:window_end], re.IGNORECASE):
                results.append(term)
    return list(set(results))  # Return unique terms

In [11]:
df_bleu_protocol=df_bleu_params.copy()
df_bleu_protocol['paper_bleu_protocol'] = df_bleu_protocol[df_bleu_protocol['paper_bleu_prelim'] == True]['paper_text'].apply(lambda x: search_terms_near_bleu(x, bleu_regex_protocol))

df_bleu_protocol["paper_bleu_protocol"].value_counts().head(20)

paper_bleu_protocol
[]                                            5297
[ngrams]                                      1271
[tokenization]                                 655
[smoothing]                                    313
[case_normalization]                           256
[precision, ngrams]                            147
[case_normalization, tokenization]             124
[smoothing, ngrams]                            123
[brevity_penalty, ngrams]                      123
[tokenization, ngrams]                         120
[brevity_penalty, precision, ngrams]            90
[smoothing, tokenization]                       83
[brevity_penalty]                               75
[case_normalization, ngrams]                    65
[case_normalization, smoothing]                 47
[clipping]                                      33
[case_normalization, smoothing, ngrams]         28
[case_normalization, tokenization, ngrams]      28
[tokenization, smoothing, ngrams]               28
[brevity_pe

#### BLEU Variants

In [12]:
bleu_regex_variants = {
    'n_gram_precision': r'\b(?:n-?gram precision|1-gram precision|2-gram precision|3-gram precision|4-gram precision)\b',
    'brevity_penalty': r'\bbrevity penalty\b|BP\b'
}

In [13]:
df_bleu_variants=df_bleu_protocol.copy()
df_bleu_variants['paper_variants'] = df_bleu_variants[df_bleu_variants['paper_bleu_prelim'] == True]['paper_text'].apply(lambda x: search_terms_near_bleu(x, bleu_regex_variants))

df_bleu_variants['paper_variants'].value_counts()

paper_variants
[]                                     8491
[brevity_penalty]                       295
[n_gram_precision]                      199
[brevity_penalty, n_gram_precision]     137
Name: count, dtype: int64

#### BLEU Packages

In [14]:
def search_for_regex_pattern(text, regex_dict):
    results = []
    for term, pattern in regex_dict.items():
        # Search for the pattern in the entire text
        if re.search(pattern, text, re.IGNORECASE):
            results.append(term)
    return list(set(results))  # Return unique terms

In [15]:
regex_bleu_versions = {
    'BLEU_original': r'\bBLEU\b.*?Papineni.*?ACL.*?2002',
    'multi_bleu': r'multi-?bleu|multi_bleu',
    'sacreBLEU': r'sacrebleu',
    'nltk_bleu': r'nltk.*?bleu',
    'mteval_v13a': r'mteval_v?13a',
    'mteval_v14': r'mteval_v?14',
    'BLEU_moses': r'bleu.*?moses',
    'BLEU_nematus': r'bleu.*?nematus',
    'BLEU_coco': r'bleu.*?coco',
    'BLEU_pytorch': r'bleu.*?pytorch',
    'BLEU_tensorflow': r'bleu.*?tensorflow',
    'BLEU_fairseq': r'fairseq.*?bleu',
    'BLEU_sacremoses': r'sacremoses.*?bleu',
    'nematus_bleu': r'nematus.*?bleu',
    'subword_nmt_bleu': r'subword-?nmt.*?bleu',
    'sentence_bleu': r'sentence-?bleu',
    'corpus_bleu': r'corpus-?bleu',
    'smoothing_bleu': r'smoothing.*?bleu',
    "coco": r'coco.*?bleu',
    "pybleu": r'pybleu|py-bleu',
    "google_bleu": r'google.*?bleu',
    "yisi_bleu": r'yisi.*?bleu',
    "bertscore_bleu": r'bertscore.*?bleu',
}

In [16]:
df_bleu_packages = df_bleu_variants.copy()
# Applying the function to the DataFrame
df_bleu_packages['paper_bleu_packages'] = df_bleu_packages[df_bleu_packages['paper_bleu_prelim'] == True]\
    ['paper_text'].apply(lambda x: search_for_regex_pattern(x, regex_bleu_versions))
    
df_bleu_packages["paper_bleu_packages"].value_counts().head(20)

paper_bleu_packages
[]                                           3786
[BLEU_moses]                                 1023
[smoothing_bleu]                              672
[google_bleu]                                 484
[smoothing_bleu, BLEU_moses]                  482
[BLEU_moses, google_bleu]                     144
[sacreBLEU]                                   144
[BLEU_pytorch]                                141
[BLEU_fairseq]                                135
[bertscore_bleu]                              112
[nltk_bleu]                                    93
[BLEU_moses, multi_bleu]                       84
[coco, BLEU_coco]                              79
[smoothing_bleu, google_bleu]                  53
[multi_bleu]                                   50
[BLEU_fairseq, sacreBLEU]                      49
[smoothing_bleu, BLEU_fairseq]                 48
[BLEU_original]                                47
[smoothing_bleu, BLEU_moses, google_bleu]      46
[BLEU_tensorflow]             

## Code Review
#### URL of code repository cited in paper

In [17]:
df_bleu_url = df_bleu_packages.copy()

# regex for codebases
regex_codebases = r'https?://(?:www\.)?(?:github\.com|gitlab\.com|bitbucket\.org|sourceforge\.net|google\.code|code\.google)[^\s)]*(?<!\.)'

# Function to extract URLs from a text
def extract_codebases(text):
    return re.findall(regex_codebases, text)

# Apply extract_codebases function to 'paper_text', store URLs in a list within each cell
df_bleu_url["code_bleu_url"] = df_bleu_url.apply(
    lambda row: extract_codebases(row['paper_text']) if row['paper_bleu_prelim'] and pd.notnull(row['paper_text']) else [],
    axis=1
)

In [20]:
df_bleu_url["code_bleu_url"].value_counts() 

code_bleu_url
[]                                                                                        72028
[https://github.com/]                                                                       175
[https://github.com/pytorch/fairseq]                                                         18
[https://github.com/moses-smt/]                                                              14
[https://github.com/facebookresearch/]                                                       13
                                                                                          ...  
[https://github.com/facebookresearch/DPR#]                                                    1
[https://github.com/cfedermann/]                                                              1
[https://github.com/moses-smt/mosesdecoder, https://github.com/facebookresearch/MIXER]        1
[https://github.com/fuzihaofzh/]                                                              1
[https://github.com/alvati

#### Does the code mention BLEU?

In [18]:
# Extract GitHub repository names from URLs
def extract_github_repo_names(urls):
    return ["/".join(urlparse(url).path.strip("/").split("/")[:2]) for url in urls if "github.com" in urlparse(url).netloc]

# Apply the function to extract GitHub repository names only if 'paper_bleu_prelim' is True
df_bleu_url['code_bleu_github'] = df_bleu_url.apply(lambda row: extract_github_repo_names(row['code_bleu_url']) if row['paper_bleu_prelim'] else [], axis=1)

In [19]:
df_bleu_url['code_bleu_github'].value_counts()

code_bleu_github
[]                          72083
[]                            182
[pytorch/fairseq]              29
[moses-smt]                    14
[moses-smt/mosesdecoder]       13
                            ...  
[SimengSun]                     1
[MIProtick]                     1
[zhijing-jin/genwiki]           1
[Marsan-Ma/chat]                1
[alvations]                     1
Name: count, Length: 841, dtype: int64

# Reproducibility

In [19]:
df_bleu_reproducible = df_bleu_url.copy()

# Initialize the 'reproducible' column as a nullable boolean
df_bleu_reproducible['reproducible'] = pd.NA

In [20]:
# R1: Check if both packages and params are not null
condition_r1 = df_bleu_reproducible['paper_bleu_packages'].notna() & df_bleu_reproducible['paper_bleu_params'].notna()
df_bleu_reproducible.loc[condition_r1, 'reproducible'] = True

df_bleu_reproducible['reproducible'].value_counts()

reproducible
True    54
Name: count, dtype: int64

In [ ]:
df_bleu_reproducible.to_pickle("bleu_paper_review.pkl")

# Export

In [ ]:
def save_dataset(df: pd.DataFrame, filepath: str = "meteorscores/data/bleu_papers.jsonl.gz") -> None:
    """
    Save the DataFrame to a .jsonl.gz file.
    
    Parameters:
    - df: The DataFrame to save.
    - filepath: The file path where the DataFrame should be saved.
    """
    try:
        df.to_json(filepath, orient="records", lines=True, compression="gzip")
        print(f"Dataset successfully saved to {filepath}")
    except Exception as e:
        print(f"Could not save dataset: {e}")

save_dataset(df_bleu_reproducible, "meteorscores/data/bleu_papers.jsonl.gz")